In [46]:
import pandas as pd
from scipy.stats import ttest_ind
import statsmodels.stats.multitest as smm

In [20]:
df = pd.read_csv('gene_high_throughput_sequencing.csv')

In [21]:
df.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [22]:
df.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

### Часть 1
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

1. для групп normal (control) и early neoplasia (treatment)
2. для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [33]:
df_normal =  df[df['Diagnosis'] == 'normal']
df_neoplasia =  df[df['Diagnosis'] == 'early neoplasia']
df_cancer =  df[df['Diagnosis'] == 'cancer']

In [34]:
ttest_ind_normal_neoplasia = []
for column in df_normal.iloc[:, 2:].columns.values:
    ttest_ind_normal_neoplasia.append(ttest_ind(df_normal[column], df_neoplasia[column], equal_var=False))

In [40]:
p_values_normal_neoplasia = [test.pvalue for test in ttest_ind_normal_neoplasia if test.pvalue < 0.05]
print "answer", len(p_values_normal_neoplasia)

answer 1575


In [41]:
ttest_ind_neoplasia_cancer = []
for column in df_neoplasia.iloc[:, 2:].columns.values:
    ttest_ind_neoplasia_cancer.append(ttest_ind(df_neoplasia[column], df_cancer[column], equal_var=False))

In [84]:
p_values_neoplasia_cancer = [test.pvalue for test in ttest_ind_neoplasia_cancer if test.pvalue < 0.05]
print "answer", len(p_values_neoplasia_cancer)

answer 3490


In [86]:
pvalues_normal_neoplasia = [test.pvalue for test in ttest_ind_normal_neoplasia]
pvalues_neoplasia_cancer = [test.pvalue for test in ttest_ind_neoplasia_cancer]

15748

### Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

In [98]:
reject1, p_corrected1, _, _ = smm.multipletests(pvalues_normal_neoplasia, alpha = 0.025, method = 'holm')
reject2, p_corrected2, _, _ = smm.multipletests(pvalues_neoplasia_cancer, alpha = 0.025, method = 'holm')

15748

In [99]:
m1=[]
n1=[]
for i in range(len(p_corrected1)):
    if ((reject1[i]==True)&(p_corrected1[i]<0.025)):
        m1.append(pvalues_normal_neoplasia[i])
        n1.append(p_corrected1[i])

def Fct(c, t) :
    if t > c :
        r = t/c
    else :
        r =  -c/t
    return r

fct1=[]
for i in range(len(m1)):
    fct1.append(Fct(m1[i],n1[i]))

c1=0
for i in range(len(fct1)):
    if abs(fct1[i])>1.5:
        c1=c1+1
    else:
        c1=c1


[15747.0, 15748.0]


In [91]:
c1

2

In [100]:
m1_2=[]
n1_2=[]

for i in range(len(p_corrected2)):
    if ((reject2[i] == True) & (p_corrected2[i] < 0.025)):
        m1_2.append(pvalues_neoplasia_cancer[i])
        n1_2.append(p_corrected2[i])
        
fct1_2=[]
for i in range(len(m1_2)):
    fct1_2.append(Fct(m1_2[i], n1_2[i]))

print fct1_2
    
c1_2=0
for i in range(len(fct1_2)):
    if abs(fct1_2[i])>1.5:
        c1_2=c1_2+1

[15708.0, 15718.0, 15675.0, 15719.0, 15673.000000000002, 15697.0, 15724.000000000002, 15737.0, 15747.0, 15748.0, 15701.0, 15696.0, 15711.000000000002, 15729.0, 15685.0, 15705.0, 15677.0, 15727.0, 15716.0, 15676.0, 15702.0, 15720.000000000002, 15682.0, 15735.0, 15687.0, 15736.0, 15678.0, 15726.0, 15734.0, 15707.0, 15721.0, 15745.0, 15680.0, 15728.000000000002, 15692.000000000002, 15679.0, 15684.000000000002, 15690.0, 15732.0, 15746.0, 15713.0, 15681.0, 15709.0, 15733.0, 15742.000000000002, 15695.0, 15698.0, 15715.0, 15670.0, 15717.0, 15739.0, 15743.0, 15689.000000000002, 15702.999999999998, 15672.000000000002, 15722.0, 15712.0, 15671.0, 15700.0, 15683.0, 15741.0, 15714.0, 15731.0, 15694.0, 15706.0, 15686.0, 15693.000000000002, 15710.0, 15674.0, 15703.999999999998, 15738.0, 15688.000000000002, 15699.0, 15740.0, 15744.000000000002, 15730.0, 15723.0, 15691.000000000002, 15725.000000000002]


In [101]:
c1_2

79

In [105]:
reject3, p_corrected3, _, _ = smm.multipletests(pvalues_normal_neoplasia, alpha = 0.025, method = 'fdr_bh') 

In [106]:
m1_3=[]
n1_3=[]
for i in range(len(p_corrected3)):
    if ((reject3[i]==True)&(p_corrected3[i]<0.025)):
        m1_3.append(pvalues_normal_neoplasia[i])
        n1_3.append(p_corrected3[i])

fct1_3=[]
for i in range(len(m1)):
    fct1_3.append(Fct(m1_3[i],n1_3[i]))

c1_3=0
for i in range(len(fct1_3)):
    if abs(fct1_3[i])>1.5:
        c1_3=c1_3+1
    else:
        c1_3=c1_3


In [107]:
print c1_3

2
